# Lab 3: Agentic Search Tools


## Setting Up the Environment

In [1]:
from dotenv import load_dotenv
import os
import sys
import json, re
import pprint
import boto3
from botocore.client import Config
import warnings

warnings.filterwarnings("ignore")
import logging

# import local modules
dir_current = os.path.abspath("")
dir_parent = os.path.dirname(dir_current)
if dir_parent not in sys.path:
    sys.path.append(dir_parent)
from utils import utils

bedrock_config = Config(
    connect_timeout=120, read_timeout=120, retries={"max_attempts": 0}
)

# Set basic configs
logger = utils.set_logger()
pp = utils.set_pretty_printer()

# Load environment variables from .env file or Secret Manager
_ = load_dotenv("../.env")
aws_region = os.getenv("AWS_REGION")
tavily_ai_api_key = utils.get_tavily_api("TAVILY_API_KEY", aws_region)


# Create a bedrock runtime client
bedrock_rt = boto3.client(
    "bedrock-runtime", region_name=aws_region, config=bedrock_config
)

# Create a bedrock client to check available models
bedrock = boto3.client("bedrock", region_name=aws_region, config=bedrock_config)


[2025-10-02 10:54:24,259] p3180 {utils.py:46} INFO - TAVILY_API_KEY variable correctly retrieved from the .env file.


In [3]:
from tavily import TavilyClient

client = TavilyClient(api_key=tavily_ai_api_key)
# run search
result = client.search("What is in Nvidia's new Blackwell GPU?", include_answer=True)

# print the answer
result['answer']

'The new Blackwell GPU features fifth-generation NVLink, enhanced Transformer Engine, and secure AI capabilities. It supports up to 576 GPUs for large-scale AI workloads.'

## Regular search


In [4]:
# choose location (try to change to your own city!)

city = "San Francisco"

query = f"""
    what is the current weather in {city}?
    Should I travel there today?
    "weather.com"
"""

> Note: search was modified to return expected results in the event of an exception. High volumes of student traffic sometimes cause rate limit exceptions.


In [5]:
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

ddg = DDGS()


def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        print(f"returning previous results due to exception reaching ddg.")
        results = [  # cover case where DDG rate limits due to high traffic volume
            "https://weather.com/weather/today/l/USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8",
        ]
        return results


for i in search(query):
    print(i)

returning previous results due to exception reaching ddg.
https://weather.com/weather/today/l/USCA0987:1:US
https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8


In [6]:
def scrape_weather_info(url):
    """Scrape content from the given URL"""
    if not url:
        return "Weather information could not be found."

    # fetch data
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    # parse result
    soup = BeautifulSoup(response.text, "html.parser")
    return soup

> Note: This produces a long output, you may want to right click and clear the cell output after you look at it briefly to avoid scrolling past it.


In [7]:
# use DuckDuckGo to find websites and take the first result
url = search(query)[0]

# scrape first wesbsite
soup = scrape_weather_info(url)

print(f"Website: {url}\n\n")
print(str(soup)[:50000])  # limit long outputs

returning previous results due to exception reaching ddg.
Website: https://weather.com/weather/today/l/USCA0987:1:US


Failed to retrieve the webpage.


In [11]:
# extract text
weather_data = []
for tag in soup.find_all(["h1", "h2", "h3", "p"]):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

# combine all elements into a single string
weather_data = "\n".join(weather_data)

# remove all spaces from the combined text
weather_data = re.sub(r"\s+", " ", weather_data)

print(f"Website: {url}\n\n")
print(weather_data)

AttributeError: 'str' object has no attribute 'find_all'

## Agentic Search


In [13]:
# run search
result = client.search(query, max_results=1)

# print first result
data = result["results"][0]["content"]

print(data)

{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1759342034, 'localtime': '2025-10-01 11:07'}, 'current': {'last_updated_epoch': 1759341600, 'last_updated': '2025-10-01 11:00', 'temp_c': 20.1, 'temp_f': 68.2, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 5.4, 'wind_kph': 8.6, 'wind_degree': 152, 'wind_dir': 'SSE', 'pressure_mb': 1017.0, 'pressure_in': 30.03, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 65, 'cloud': 75, 'feelslike_c': 20.1, 'feelslike_f': 68.2, 'windchill_c': 19.9, 'windchill_f': 67.8, 'heatindex_c': 19.9, 'heatindex_f': 67.8, 'dewpoint_c': 13.9, 'dewpoint_f': 57.0, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.4, 'gust_mph': 7.7, 'gust_kph': 12.4}}


In [15]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
parsed_json = json.loads(data.replace("'", '"'))

# pretty print JSON with syntax highlighting
formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(
    formatted_json, lexers.JsonLexer(), formatters.TerminalFormatter()
)

print(colorful_json)

{
    "location": {
        "name": "San Francisco",
        "region": "California",
        "country": "United States of America",
        "lat": 37.775,
        "lon": -122.4183,
        "tz_id": "America/Los_Angeles",
        "localtime_epoch": 1759342034,
        "localtime": "2025-10-01 11:07"
    },
    "current": {
        "last_updated_epoch": 1759341600,
        "last_updated": "2025-10-01 11:00",
        "temp_c": 20.1,
        "temp_f": 68.2,
        "is_day": 1,
        "condition": {
            "text": "Partly cloudy",
            "icon": "//cdn.weatherapi.com/weather/64x64/day/116.png",
            "code": 1003
        },
        "wind_mph": 5.4,
        "wind_kph": 8.6,
        "wind_degree": 152,
        "wind_dir": "SSE",
        "pressure_mb": 1017.0,
        "pressure_in": 30.03,
        "precip_mm": 0.0,
        "precip_in": 0.0,
        "humidity": 65,
        "cloud": 75,
        "feelslike_c": 20.1,
        "feelslike_f": 68.2,
        "windchill_c": 19.9,
     